In [3]:
from ib_insync import *
util.startLoop()  

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=9)

<IB connected to 127.0.0.1:7497 clientId=9>

In [5]:
contracts = [Forex(pair) for pair in ('EURUSD', 'USDJPY', 'GBPUSD', 'USDCHF', 'USDCAD', 'AUDUSD')]
ib.qualifyContracts(*contracts)

eurusd = contracts[0]

In [6]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

In [37]:
help(ib.reqMktData)

Help on method reqMktData in module ib_insync.ib:

reqMktData(contract:ib_insync.contract.Contract, genericTickList:str='', snapshot:bool=False, regulatorySnapshot:bool=False, mktDataOptions:List[ib_insync.contract.TagValue]=None) -> ib_insync.ticker.Ticker method of ib_insync.ib.IB instance
    Subscribe to tick data or request a snapshot.
    Returns the Ticker that holds the market data. The ticker will
    initially be empty and gradually (after a couple of seconds)
    be filled.
    
    https://interactivebrokers.github.io/tws-api/md_request.html
    
    Args:
        contract: Contract of interest.
        genericTickList: Comma separated IDs of desired
            generic ticks that will cause corresponding Ticker fields
            to be filled:
    
            =====  ================================================
            ID     Ticker fields
            =====  ================================================
            100    ``putVolume``, ``callVolume`` (for optio

In [7]:
ticker = ib.ticker(eurusd)
ib.sleep(2)

ticker

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2020, 3, 23, 22, 45, 51, 728592, tzinfo=datetime.timezone.utc), bid=1.07503, bidSize=2000000, ask=1.0751, askSize=2000000, prevBid=1.07504, prevBidSize=1000000, prevAsk=1.07512, prevAskSize=1000000, high=1.0752, low=1.072, close=1.0725, halted=0.0, ticks=[TickData(time=datetime.datetime(2020, 3, 23, 22, 45, 51, 728592, tzinfo=datetime.timezone.utc), tickType=2, price=1.0751, size=2000000)])

In [8]:
ticker.marketPrice()

1.0750449999999998

In [15]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(
    index=[c.pair() for c in contracts],
    columns=['bidSize', 'bid', 'ask', 'askSize', 'high', 'low', 'close'])

def onPendingTickers(tickers):
    for t in tickers:
        df.loc[t.contract.pair()] = (
            t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait=True)
    display(df)        

ib.pendingTickersEvent += onPendingTickers
ib.sleep(30)
ib.pendingTickersEvent -= onPendingTickers

,bidSize,bid,ask,askSize,high,low,close
EURUSD,1000000,1.07465,1.07469,1000000,1.0752,1.072,1.0725
USDJPY,1000000,110.888,110.89,1000000,111.325,110.865,111.25
GBPUSD,2000000,1.1555,1.15599,1000000,1.15645,1.14855,1.1546
USDCHF,1000000,0.98362,0.9839,1000000,0.98505,0.98335,0.9848
USDCAD,1000000,1.44819,1.44833,1000000,1.45335,1.4478,1.4506
AUDUSD,2000000,0.58376,0.5839,1000000,0.58395,0.58125,0.58255


In [39]:
help(ib.pendingTickersEvent)

Help on Event in module eventkit.event object:

class Event(builtins.object)
 |  Enable event passing between loosly coupled components.
 |  The event emits values to connected listeners and has
 |  a selection of operators to create general data flow pipelines.
 |  
 |  Args:
 |      name: Name to use for this event.
 |  
 |  Methods defined here:
 |  
 |  __aiter__ = aiter(self, skip_to_last:bool=False, tuples:bool=False)
 |  
 |  __await__(self)
 |      Asynchronously await the next emit of an event::
 |      
 |          async def coro():
 |              args = await event
 |              ...
 |      
 |      If the event does an empty ``emit()``, then the value
 |      of ``args`` is set to ``util.NO_VALUE``.
 |      
 |      :meth:`wait` and :meth:`__await__` are each other's inverse.
 |  
 |  __bool__(self)
 |  
 |  __call__ = emit(self, *args)
 |  
 |  __contains__(self, c)
 |      See if callable is already connected.
 |  
 |  __getitem__(self, fork_targets) -> 'Fork'
 |  
 | 

In [12]:
ticker = ib.reqTickByTickData(eurusd, 'BidAsk')
ib.sleep(2)
print(ticker)

ib.cancelTickByTickData(ticker.contract, 'BidAsk')

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2020, 3, 23, 22, 48, 49, 941014, tzinfo=datetime.timezone.utc), bid=1.07438, bidSize=1000000, ask=1.07442, askSize=1000000, prevBid=1.07439, prevBidSize=2000000, prevAsk=1.0744, prevAskSize=2000000, high=1.0752, low=1.072, close=1.0725, halted=0.0, tickByTicks=[TickByTickBidAsk(time=datetime.datetime(2020, 3, 23, 22, 48, 49, 941014, tzinfo=datetime.timezone.utc), bidPrice=1.07438, askPrice=1.07442, bidSize=1000000, askSize=1000000, tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False))])


In [13]:
help(ticker)

Help on Ticker in module ib_insync.ticker object:

class Ticker(builtins.object)
 |  Current market data such as bid, ask, last price, etc. for a contract.
 |  
 |  Streaming level-1 ticks of type :class:`.TickData` are stored in
 |  the ``ticks`` list.
 |  
 |  Streaming level-2 ticks of type :class:`.MktDepthData` are stored in the
 |  ``domTicks`` list. The order book (DOM) is available as lists of
 |  :class:`.DOMLevel` in ``domBids`` and ``domAsks``.
 |  
 |  Streaming tick-by-tick ticks are stored in ``tickByTicks``.
 |  
 |  For options the :class:`.OptionComputation` values for the bid, ask, resp.
 |  last price are stored in the ``bidGreeks``, ``askGreeks`` resp.
 |  ``lastGreeks`` attributes. There is also ``modelGreeks`` that conveys
 |  the greeks as calculated by Interactive Brokers' option model.
 |  
 |  Events:
 |      * ``updateEvent`` (ticker: :class:`.Ticker`)
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __hash__(se

In [14]:
import datetime

start = ''
end = datetime.datetime.now()
ticks = ib.reqHistoricalTicks(eurusd, start, end, 1000, 'BID_ASK', useRth=False)

ticks[-1]

HistoricalTickBidAsk(time=datetime.datetime(2020, 3, 23, 22, 50, 12, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False), priceBid=1.07448, priceAsk=1.07455, sizeBid=1000000, sizeAsk=1000000)

In [26]:
contract = Forex('EURUSD')
ib.qualifyContracts(contract)

order = LimitOrder('SELL', 20000, 1.08)

In [27]:
trade = ib.placeOrder(contract, order)

In [34]:
trade.log

[TradeLogEntry(time=datetime.datetime(2020, 3, 23, 22, 59, 41, 587535, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''),
 TradeLogEntry(time=datetime.datetime(2020, 3, 23, 22, 59, 41, 647541, tzinfo=datetime.timezone.utc), status='Submitted', message='')]

In [35]:
help(trade)

Help on Trade in module ib_insync.order object:

class Trade(builtins.object)
 |  Trade keeps track of an order, its status and all its fills.
 |  
 |  Events:
 |      * ``statusEvent`` (trade: :class:`.Trade`)
 |      * ``modifyEvent`` (trade: :class:`.Trade`)
 |      * ``fillEvent`` (trade: :class:`.Trade`, fill: :class:`.Fill`)
 |      * ``commissionReportEvent`` (trade: :class:`.Trade`,
 |        fill: :class:`.Fill`, commissionReport: :class:`.CommissionReport`)
 |      * ``filledEvent`` (trade: :class:`.Trade`)
 |      * ``cancelEvent`` (trade: :class:`.Trade`)
 |      * ``cancelledEvent`` (trade: :class:`.Trade`)
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |  
 |  __init__(self, contract:ib_insync.contract.Contract=<factory>, order:ib_insync.order.Order=<factory>, orderStatus:'OrderStatus'=<factory>, fills:List[ib_insync.objects.Fill]=<factory>, log:List[ib_insync.objects.TradeLogEntry]=<factory>) -> None
 |  
 |  __post_init__(self)
 |  
 |  __repr__(self)
 |  

In [36]:
dir()

['AccountValue',
 'Bag',
 'BarData',
 'BarDataList',
 'BarList',
 'Bond',
 'BracketOrder',
 'CFD',
 'Client',
 'ComboLeg',
 'CommissionReport',
 'Commodity',
 'ConnectionStats',
 'ContFuture',
 'Contract',
 'ContractDescription',
 'ContractDetails',
 'DOMLevel',
 'DeltaNeutralContract',
 'DepthMktDataDescription',
 'Dividends',
 'Event',
 'Execution',
 'ExecutionCondition',
 'ExecutionFilter',
 'FamilyCode',
 'Fill',
 'FlexError',
 'FlexReport',
 'Forex',
 'FundamentalRatios',
 'Future',
 'FuturesOption',
 'HistogramData',
 'HistoricalNews',
 'HistoricalTick',
 'HistoricalTickBidAsk',
 'HistoricalTickLast',
 'IB',
 'IBC',
 'IBController',
 'In',
 'Index',
 'LimitOrder',
 'MarginCondition',
 'MarketOrder',
 'MktDepthData',
 'MutualFund',
 'NewsArticle',
 'NewsBulletin',
 'NewsProvider',
 'NewsTick',
 'Option',
 'OptionChain',
 'OptionComputation',
 'Order',
 'OrderComboLeg',
 'OrderCondition',
 'OrderState',
 'OrderStatus',
 'Out',
 'PercentChangeCondition',
 'PnL',
 'PnLSingle',
 'Port